Ref:<br>
    [1] Python: Design Patterns, LinkedIn Learning

## Introduction

Design pattern types:
- Creational:
 - use to create objects systematicaly
 - benefit: flexibility
 - polymorphism
- Structural
 - use: establish relationship between components
 - inheritance
- Behavioral
 - use: object interact with each other
 - focus: protocols
 - methods
 
 
OOP
Class defines objects using 
 - attributes: Property of entity. current state
 - behaviors: methods
 
Inheritance: relationship between classes and childs <br>
polymorphism: relies on inheritance and allows child classes to be instantialed and treated as the same type as their parent


pattern context: scenario in which we choose to use a pattern
- participants: classes involved to form a design pattern
- Quality Attributes: Non-functional requirements (usabilty, reliability ...)
- Forces: trade-offs (quality attributes create forces)
- Consequences:


## Creational Pattern

### Factory
Object creating other objects <br>
not sure which type of object (class) we need at runtime <br>

In [2]:
class Dog:
    def __init__(self, name):
        self._name = name
    def speak(self):
        return "woof!"
class Cat:
    def __init__(self, name):
        self._name = name
    def speak(self):
        return "meow!"

    
def get_pet(pet="dog"):
    """
    Factory method
    """
    pets = dict(dog=Dog("Goli"), cat=Cat("Lili"))
    return pets[pet]

d = get_pet("dog")
print(d.speak())
c = get_pet("cat")
print(c.speak())

woof!
meow!


### Abstract Factory
User expectation yields **multiple, related** objects but does not know which family it is until runtime

elements:
- abstract factory: pet factory
- concrete factory: dog factory and cat factory (mostly singleton)
- abstract product: 
- concrete product: dog and dog food, cat and cat food

in python we implement abstract factory without inheritance, because python is dynamically typed language

In [3]:
class Dog:
    """
    one of the objects to be returned
    """
    def speak(self):
        return "woof!"
    def __str__(self):
        return "Dog"
    
class DogFactory:
    """
    concrete factory 
    """
    def get_pet(self):
        """returns dog object (concrete product)"""
        return Dog()
    def get_food(self):
        """returns dog food object (concrete product)"""
        return "Dog Food!"
class PetStore:
    """Our Abstract Factory"""
    def __init__(self, pet_factory=None):
        self._pet_factory = pet_factory
    def show_pet(self):
        pet = self._pet_factory.get_pet()
        pet_food = self._pet_factory.get_food()
        
        print(f"Our pet is {pet}")
        print(f"Our pet says {pet.speak()}")
        print(f"Our pet eats {pet_food}")
#
# create a concrete factory
factory = DogFactory()

# Abstract factory -> gets a concrete factory
shop = PetStore(factory)

# Invoke utility methods
shop.show_pet()

Our pet is Dog
Our pet says woof!
Our pet eats Dog Food!


### Singleton
Allow only one object to be created from a class. <br>
It is object oriented way of defining globals <br>
example usage: cache information and use among objects. <br>

singleton uses inheritance <br>
shares an attribute among all instances <br>


In [4]:
class Borg:
    """Borg design pattern"""
    _shared_data = {} # Attribute dictionary
    def __init__(self):
        self.__dict__ = self._shared_data # makes _shared_data attribute dict

class Singleton(Borg):
    """Singleton Class"""
    def __init__(self, **kwargs):
        Borg.__init__(self)
        self._shared_data.update(kwargs)
    def __str__(self):
        return str(self._shared_data)
    
# create a singleton object
x = Singleton(HTTP="Hyper Text Transfer Protocol")
# print object
print(x)

#
y = Singleton(SNMP="Simple Network Management Protocol")
print(y)
print(x)


{'HTTP': 'Hyper Text Transfer Protocol'}
{'HTTP': 'Hyper Text Transfer Protocol', 'SNMP': 'Simple Network Management Protocol'}
{'HTTP': 'Hyper Text Transfer Protocol', 'SNMP': 'Simple Network Management Protocol'}


### Builder

solution to anti-pattern called telescoping constructor (build using exessive number of constructors). <br>
use partitions:
- director: in charge of building the product
- abstract builder: create all required interfaces required in buidling an object
- concrete builder: inherits from the abstract builder and implements the interface for a specific type of product
- product: is the object being built

Builder patter does not rely on polymorphism.

In [6]:
class Director():
    """Director"""
    def __init__(self, builder):
        self._builder = builder
        
    def construct_car(self):
        self._builder.create_new_car()
        self._builder.add_model()
        self._builder.add_tires()
        self._builder.add_engine()
    def get_car(self):
        return self._builder.car

class Car():
    """Product"""
    def __init__(self):
        self.model =None
        self.tires = None
        self.engine = None
    def __str__(self):
        return f'{self.model} | {self.tires} | {self.engine}'
    

class Builder():
    """Abstract Builder"""
    def __init__(self):
        self.car = None
    def create_new_car(self):
        self.car = Car()

class SkyLarkBuilder(Builder):
    """Concrete Builder"""
    def add_model(self):
        self.car.model = "skylark"
    def add_tires(self):
        self.car.tires = "regular tire"
    def add_engine(self):
        self.car.engine = "turbo"
        
builder = SkyLarkBuilder()
director = Director(builder)
director.construct_car()
print(director.get_car())

skylark | regular tire | turbo


### Prototype

we are talking about copying instead of creating. Creating many identical object is expensive but cloning makes it less expensive. <br>
steps:
- create a prototypical instance
- clone it when need a replica

abstract factory is related to this pattern

In [42]:
import copy

class Prototype:
    
    def __init__(self):
        # creat a dict to be cloned
        self._objects = {}
        
    def register_object(self, name, obj):
        """Register an object"""
        self._objects[name] = obj
        
    def unregister_object(self, name):
        """Unregister an object"""
        del self._objects[name]
        
        
    def clone(self, name, **attr):
        """Clone a registered object and update its attributes"""
        obj = copy.deepcopy(self._objects.get(name))
        obj.__dict__.update(attr)
        return obj
        
        
class Car:
    def __init__(self):
        self.name = "Skylark"
        self.color = "Red"
        self.options = "Ex"
        
    def __str__(self):
        return '{} | {} | {}'.format(self.name, self.color, self.options)
        
c = Car()
print(c)
p = Prototype()
p.register_object('skylark',c)
c1=p.clone('skylark')
print(c1)
c2=p.clone('skylark', color='black')
print(c2)

Skylark | Red | Ex
Skylark | Red | Ex
Skylark | black | Ex


## Structural Patterns

### Decorator
allows users to add new features to existing object without changing the object

In [43]:
from functools import wraps

def make_blink(function):
    """Defines the decorator"""
    #This makes the decorator transparent in terms of its name and docstring
    @wraps(function)
    
    #Define the inner function
    def decorator():
        # Grab the return value of the function being decorated
        ret = function()
        
        # Add new functionality to the function being decorated
        return "<blink>" + ret + "</blink>"
    return decorator

#Apply the decorator here!
@make_blink
def hello_world():
    """Original function! """

    return "Hello, World!"

#Check the result of decorating
print(hello_world())

#Check if the function name is still the same name of the function being decorated
print(hello_world.__name__)

#Check if the docstring is still the same as that of the function being decorated
print(hello_world.__doc__)


<blink>Hello, World!</blink>
hello_world
Original function! 


### Proxy
Is used to postpone the **object creation** as late as possible because it requires high resources. <br>
so we need a placeholder, and create the object when it is absoloutly necessary 

In [44]:
import time

class Producer:
    """Define the 'resource-intensive' object to instantiate!"""
    def produce(self):
        print("Producer is working hard!")

    def meet(self):
        print("Producer has time to meet you now!")

class Proxy:
    """"Define the 'relatively less resource-intensive' proxy to instantiate as a middleman"""
    def __init__(self):  
        self.occupied = 'No'
        self.producer = None
    
    def produce(self):
        """Check if Producer is available"""
        print("Artist checking if Producer is available ...")

        if self.occupied == 'No':
            #If the producer is available, create a producer object!
            self.producer = Producer()
            time.sleep(2)


            #Make the prodcuer meet the guest!
            self.producer.meet()

        else:
            #Otherwise, don't instantiate a producer 
            time.sleep(2)
            print("Producer is busy!")

#Instantiate a Proxy
p = Proxy()
#Make the proxy: Artist produce until Producer is available
p.produce()

#Change the state to 'occupied'
p.occupied = 'Yes'

#Make the Producer produce
p.produce()


Artist checking if Producer is available ...
Producer has time to meet you now!
Artist checking if Producer is available ...
Producer is busy!


### Adapter
converts the interface of a class into another one a client is expecting <br>
bridges and decorators are related to adapter pattern <br>

In [46]:
class Korean:
    """Korean speaker"""
    def __init__(self):
        self.name = "Korean"

    def speak_korean(self):
        return "An-neyong?"

class British:
    """English speaker"""
    def __init__(self):
        self.name = "British"

    #Note the different method name here!
    def speak_english(self):
        return "Hello!"

class Adapter:
    """This changes the generic method name to individualized method names"""

    def __init__(self, object, **adapted_method):
        """Change the name of the method"""
        self._object = object

        #Add a new dictionary item that establishes the mapping between the generic method name: speak() and the concrete method
        #For example, speak() will be translated to speak_korean() if the mapping says so
        self.__dict__.update(adapted_method)

    def __getattr__(self, attr):
        """Simply return the rest of attributes!"""
        return getattr(self._object, attr)

#List to store speaker objects
objects = []

#Create a Korean object
korean = Korean()

#Create a British object
british = British()

#Append the objects to the objects list
objects.append(Adapter(korean, speak=korean.speak_korean))
objects.append(Adapter(british, speak=british.speak_english))


for obj in objects:
    print("{} says '{}'\n".format(obj.name, obj.speak()))


Korean says 'An-neyong?'

British says 'Hello!'



### Composite
a tree structure is used to represent part-whole relationship <br>
elements:
- Component
- Child
- Composite

In [48]:
class Component(object):
    """
    Abstract class
    here we define our interface method
    """
    

    def __init__(self, *args, **kwargs):
        pass

    def component_function(self):
        pass

class Child(Component): #Inherits from the abstract class, Component
    """Concrete class"""

    def __init__(self, *args, **kwargs):
        Component.__init__(self, *args, **kwargs)

        #This is where we store the name of your child item!
        self.name = args[0]

    def component_function(self):
        #Print the name of your child item here!
        print("{}".format(self.name))

class Composite(Component): #Inherits from the abstract class, Component
    """Concrete class and maintains the tree recursive structure"""

    def __init__(self, *args, **kwargs):
        Component.__init__(self, *args, **kwargs)

        #This is where we store the name of the composite object
        self.name = args[0]

        #This is where we keep our child items
        self.children = []

    def append_child(self, child):
        """Method to add a new child item"""
        self.children.append(child)

    def remove_child(self, child):
        """Method to remove a child item"""
        self.children.remove(child)

    def component_function(self):

        #Print the name of the composite object
        print("{}".format(self.name))

        #Iterate through the child objects and invoke their
        # component function printing their names
        for i in self.children:
            i.component_function()

#Build a composite submenu 1
sub1 = Composite("submenu1")

#Create a new child sub_submenu 11
sub11 = Child("sub_submenu 11")
#Create a new Child sub_submenu 12
sub12 = Child("sub_submenu 12")

#Add the sub_submenu 11 to submenu 1
sub1.append_child(sub11)
#Add the sub_submenu 12 to submenu 1
sub1.append_child(sub12)

#Build a top-level composite menu
top = Composite("top_menu")

#Build a submenu 2 that is not a composite
sub2 = Child("submenu2")

#Add the composite submenu 1 to the top-level composite menu
top.append_child(sub1)
#Add the plain submenu 2 to the top-level composite menu
top.append_child(sub2)

#Let's test if our Composite pattern works!
top.component_function()


top_menu
submenu1
sub_submenu 11
sub_submenu 12
submenu2


### Bridge
problem:
- unrelated, parallel or orthogonal abstraction
- one is implementation specific and one is implementation independent

abstract factory and adapter are related to this pattern

In [49]:
class DrawingAPIOne(object):
	"""Implementation-specific abstraction: concrete class one"""
	def draw_circle(self, x, y, radius):
		print("API 1 drawing a circle at ({}, {} with radius {}!)".format(x, y, radius))


class DrawingAPITwo(object):
	"""Implementation-specific abstraction: concrete class two"""
	def draw_circle(self, x, y, radius):
		print("API 2 drawing a circle at ({}, {} with radius {}!)".format(x, y, radius))

class Circle(object):
	"""Implementation-independent abstraction: for example, there could be a rectangle class!"""

	def __init__(self, x, y, radius, drawing_api):
		"""Initialize the necessary attributes"""
		self._x = x
		self._y = y
		self._radius = radius
		self._drawing_api = drawing_api

	def draw(self):
		"""Implementation-specific abstraction taken care of by another class: DrawingAPI"""
		self._drawing_api.draw_circle(self._x, self._y, self._radius)

	def scale(self, percent):
		"""Implementation-independent"""
		self._radius *= percent


#Build the first Circle object using API One
circle1 = Circle(1, 2, 3, DrawingAPIOne())
#Draw a circle
circle1.draw()

#Build the second Circle object using API Two
circle2 = Circle(2, 3, 4, DrawingAPITwo())
#Draw a circle
circle2.draw()



API 1 drawing a circle at (1, 2 with radius 3!)
API 2 drawing a circle at (2, 3 with radius 4!)


## Behavioral Patterns

### Observer
Establish one to many between a subject and multiple observers<br>
subject needs to be monitored and another need to be notified if the subject is modified

Solution:
- Subject: allows attach, detach and notify observer
- Conceret subject: inherit from abstract subject

singleton is related

In [51]:
class Subject(object): #Represents what is being 'observed'

    def __init__(self):
        self._observers = [] # This where references to all the observers are being kept
                             # Note that this is a one-to-many relationship: there will be one subject to be observed by multiple _observers

    def attach(self, observer):
        if observer not in self._observers: #If the observer is not already in the observers list
            self._observers.append(observer) # append the observer to the list

    def detach(self, observer): #Simply remove the observer
        try:
            self._observers.remove(observer)
        except ValueError:
            pass

    def notify(self, modifier=None):
        for observer in self._observers: # For all the observers in the list
            if modifier != observer: # Don't notify the observer who is actually updating the temperature 
                observer.update(self) # Alert the observers!

class Core(Subject): #Inherits from the Subject class

    def __init__(self, name=""):
        Subject.__init__(self)
        self._name = name #Set the name of the core
        self._temp = 0 #Initialize the temperature of the core

    @property #Getter that gets the core temperature
    def temp(self):
        return self._temp

    @temp.setter #Setter that sets the core temperature
    def temp(self, temp):
        self._temp = temp
        self.notify() #Notify the observers whenever somebody changes the core temperature

class TempViewer:

    def update(self, subject): #Alert method that is invoked when the notify() method in a concrete subject is invoked
        print("Temperature Viewer: {} has Temperature {}".format(subject._name, subject._temp))

#Let's create our subjects
c1 = Core("Core 1")
c2 = Core("Core 2")

#Let's create our observers
v1 = TempViewer()
v2 = TempViewer()

#Let's attach our observers to the first core
c1.attach(v1)
c1.attach(v2)

#Let's change the temperature of our first core
c1.temp = 80
c1.temp = 90


Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 80
Temperature Viewer: Core 1 has Temperature 90
Temperature Viewer: Core 1 has Temperature 90


### Visitor
add new feature to a class hierarchy with minimal changes

In [52]:
class House(object): #The class being visited 
    def accept(self, visitor):
        """Interface to accept a visitor"""
        visitor.visit(self) #Triggers the visiting operation!

    def work_on_hvac(self, hvac_specialist):
        print(self, "worked on by", hvac_specialist) #Note that we now have a reference to the HVAC specialist object in the house object!

    def work_on_electricity(self, electrician):
        print(self, "worked on by", electrician) #Note that we now have a reference to the electrician object in the house object!

    def __str__(self):
        """Simply return the class name when the House object is printed"""
        return self.__class__.__name__


class Visitor(object):
    """Abstract visitor"""
    def __str__(self):
        """Simply return the class name when the Visitor object is printed"""
        return self.__class__.__name__


class HvacSpecialist(Visitor): #Inherits from the parent class, Visitor
    """Concrete visitor: HVAC specialist"""
    def visit(self, house):
        house.work_on_hvac(self) #Note that the visitor now has a reference to the house object


class Electrician(Visitor): #Inherits from the parent class, Visitor
    """Concrete visitor: electrician"""
    def visit(self, house):
        house.work_on_electricity(self) #Note that the visitor now has a reference to the house object

#Create an HVAC specialist
hv = HvacSpecialist()
#Create an electrician
e = Electrician()

#Create a house
home = House()

#Let the house accept the HVAC specialist and work on the house by invoking the visit() method
home.accept(hv)

#Let the house accept the electrician and work on the house by invoking the visit() method
home.accept(e)




House worked on by HvacSpecialist
House worked on by Electrician


### Iterator
related to composite design pattern

In [53]:
def count_to(count):
    """Our iterator implementation"""
     
    #Our list
    numbers_in_german = ["eins", "zwei", "drei", "vier", "funf"]

    #Our built-in iterator
    #Creates a tuple such as (1, "eins")
    iterator = zip(range(count), numbers_in_german)
     
    #Iterate through our iterable list
    #Extract the German numbers
    #Put them in a generator called number
    for position, number in iterator:
         
        #Returns a 'generator' containing numbers in German
        yield number 

#Let's test the generator returned by our iterator
for num in count_to(3):
    print("{}".format(num))

for num in count_to(4):
    print("{}".format(num))
     

eins
zwei
drei
eins
zwei
drei
vier


### Strategy
need for dynamically changing behavior of a class <br>
we need to use types module

In [54]:
import types #Import the types module

class Strategy:
    """The Strategy Pattern class"""
    
    def __init__(self, function=None):
        self.name = "Default Strategy"
        
        #If a reference to a function is provided, replace the execute() method with the given function
        if function:
            self.execute = types.MethodType(function, self)
            
    def execute(self): #This gets replaced by another version if another strategy is provided.
        """The defaut method that prints the name of the strategy being used"""
        print("{} is used!".format(self.name))

#Replacement method 1
def strategy_one(self):
    print("{} is used to execute method 1".format(self.name))

#Replacement method 2    
def strategy_two(self):
    print("{} is used to execute method 2".format(self.name))
    
#Let's create our default strategy
s0 = Strategy()
#Let's execute our default strategy
s0.execute()

#Let's create the first varition of our default strategy by providing a new behavior
s1 = Strategy(strategy_one)
#Let's set its name
s1.name = "Strategy One"
#Let's execute the strategy
s1.execute()

s2 = Strategy(strategy_two)
s2.name = "Strategy Two"
s2.execute()


Default Strategy is used!
Strategy One is used to execute method 1
Strategy Two is used to execute method 2


### Chain of Responsibility
decouples request from its processing
solution:
- abstract handler: stores a successor that will handle a request if the current handler doesn't handle it. 
- concrete handler: Concrete handlers check if they can handle the request. If they can, they handle it and return a true value, indicating that the request was handled.

Composite is related to the chain of responsibility, design pattern.

In [56]:
class Handler: #Abstract handler
    """Abstract Handler"""
    def __init__(self, successor):
        self._successor = successor # Define who is the next handler

    def handle(self, request):
            handled = self._handle(request) #If handled, stop here

            #Otherwise, keep going
            if not handled:
                self._successor.handle(request)    

    def _handle(self, request):
        raise NotImplementedError('Must provide implementation in subclass!')

class ConcreteHandler1(Handler): # Inherits from the abstract handler
    """Concrete handler 1"""
    def _handle(self, request):
        if 0 < request <= 10: # Provide a condition for handling
            print("Request {} handled in handler 1".format(request))
            return True # Indicates that the request has been handled

class DefaultHandler(Handler): # Inherits from the abstract handler
    """Default handler"""

    def _handle(self, request):
        """If there is no handler available"""
        #No condition checking since this is a default handler
        print("End of chain, no handler for {}".format(request))
        return True # Indicates that the request has been handled

class Client: # Using handlers
    def __init__(self):
        self.handler = ConcreteHandler1(DefaultHandler(None)) # Create handlers and use them in a sequence you want
                                                              # Note that the default handler has no successor

    def delegate(self, requests): # Send your requests one at a time for handlers to handle
        for request in requests:
                self.handler.handle(request)

# Create a client
c = Client()

# Create requests
requests = [2, 5, 30]

# Send the requests
c.delegate(requests)



Request 2 handled in handler 1
Request 5 handled in handler 1
End of chain, no handler for 30
